In [1]:
import pandas as pd
import numpy as np 
from itertools import permutations

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#generate random data 
data = np.random.randint(10, size=(4, 2))
labels = ['A', 'B', 'C','D']
df = pd.DataFrame(data, columns=['X', 'Y'], index=labels)

#convert to a dictionary 
data_dict = {}
for idx, col in df.iterrows(): 
    data_dict[idx] = col.values
    
#data_dict   

In [3]:
#calculate the manhattan distance of our data values + put it into a n x n distance matrix 
def manhattan_dist(x, y):
    return sum(abs(a-b) for a,b in zip(x,y))

In [4]:
#put distances and labels into a dictionary
def build_tree():
    D = {}
    for label, val in data_dict.items():
        D[label] = {}
        for label2, val2 in data_dict.items():
            D[label][label2] = manhattan_dist(val, val2) 
    return D

In [5]:
tree = build_tree()
tree

{'A': {'A': 0, 'B': 6, 'C': 6, 'D': 8},
 'B': {'A': 6, 'B': 0, 'C': 8, 'D': 2},
 'C': {'A': 6, 'B': 8, 'C': 0, 'D': 10},
 'D': {'A': 8, 'B': 2, 'C': 10, 'D': 0}}

In [6]:
#visualize as n x n distance matrix         
dm = pd.DataFrame.from_dict(tree)
dm

,A,B,C,D
A,0,6,6,8
B,6,0,8,2
C,6,8,0,10
D,8,2,10,0


In [7]:
#check if the matrix is additive using four point condition
def is_additive(matrix, labels):
    i = labels[0]
    j = labels[1] 
    k = labels[2] 
    l = labels[3] 
    result = ''
    if ((matrix[i][j] + matrix[k][l] <= matrix[i][k] + matrix[j][l]) and 
    (matrix[i][k] + matrix[j][l] == matrix[i][l] + matrix[j][k])):
        result = "Additive"
    else:
        result = "Not additive"
    return result 

In [8]:
cols = 'ABCD'
labels = permutations(cols,4)

In [9]:
check = []
for combs in labels:
    new_label = []
    for item in combs:
        new_label+=item
    check.append(is_additive(dm, new_label))
check

['Not additive',
 'Not additive',
 'Additive',
 'Additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Additive',
 'Additive',
 'Additive',
 'Additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Not additive',
 'Additive',
 'Additive',
 'Not additive',
 'Not additive']

## Niegbor join

In [10]:
import copy
clusters = []
for cluster in list(dm):
    clusters.append(cluster)
cluster2idx = { name : idx for idx, name in enumerate(clusters) }
idx2cluster = copy.deepcopy(clusters)
m = copy.deepcopy(dm).to_dict()
Z = np.empty((0,4), float)
result = ""
print(clusters)
#membership = [set([cluster2idx[name]]) for name in clusters]

['A', 'B', 'C', 'D']


In [11]:
M, N = dm.shape
new_cluster_idx = M+1
while len(clusters) > 1:
    print(result)
    print(m)
    idx1 = None
    idx2 = None
    ui = 0
    uj = 0
    Sd = 100000
    for i in clusters:
        for j in clusters:
            if i != j:
                ui = sum(m[i][k] for k in clusters)
                uj = sum(m[j][k] for k in clusters)
                curr = (M - 2) * m[i][j] - ui - uj
                if Sd > curr:
                    Sd = curr
                    idx1 = cluster2idx[i]
                    idx2 = cluster2idx[j]
    
    Dij = m[idx2cluster[idx1]][idx2cluster[idx2]]
    new_cluster = "(" + idx2cluster[idx1] +"." + idx2cluster[idx2] + ")"
    result = new_cluster
    m[new_cluster] = {}
    new_cluster_idx += 1
    #membership.append(membership[cluster2idx[idx1]] | membership[cluster2idx[idx2]])
    #Z = np.append(Z, np.array([[idx1, idx2, Sd]]))
    
    m[idx2cluster[idx1]][new_cluster] = (m[idx2cluster[idx1]][idx2cluster[idx2]] + (1/(M-2)) * (ui - uj))/2
    m[idx2cluster[idx2]][new_cluster] = (m[idx2cluster[idx1]][idx2cluster[idx2]] + (1/(M-2)) * (uj - ui))/2
    
    #if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
    #if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]
    
    clusters.remove(idx2cluster[idx1])
    clusters.remove(idx2cluster[idx2])
    
    m[new_cluster][new_cluster] = 0
    for c in clusters:
        m[c][new_cluster] = (m[c][idx2cluster[idx1]] + m[c][idx2cluster[idx2]] - Dij)/2
        m[new_cluster][c] = m[c][new_cluster]
        for k in list(m[c].keys()):
            if k == idx2cluster[idx1] or k == idx2cluster[idx2]:
                del m[c][k]
        
    del cluster2idx[idx2cluster[idx1]]
    del cluster2idx[idx2cluster[idx2]]
    if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
    if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]
    
    clusters.append(new_cluster)
    cluster2idx[new_cluster] = len(idx2cluster)
    idx2cluster.append(new_cluster)
        
    #M -= 1
    #N -= 1


{'A': {'A': 0, 'B': 6, 'C': 6, 'D': 8}, 'B': {'A': 6, 'B': 0, 'C': 8, 'D': 2}, 'C': {'A': 6, 'B': 8, 'C': 0, 'D': 10}, 'D': {'A': 8, 'B': 2, 'C': 10, 'D': 0}}
(A.C)
{'B': {'B': 0, 'D': 2, '(A.C)': 4.0}, 'D': {'B': 2, 'D': 0, '(A.C)': 6.0}, '(A.C)': {'(A.C)': 0, 'B': 4.0, 'D': 6.0}}
(B.D)
{'(A.C)': {'(A.C)': 0, '(B.D)': 4.0}, '(B.D)': {'(B.D)': 0, '(A.C)': 4.0}}


In [12]:
print(result)

((A.C).(B.D))
